# 엑셀파일 읽어오기

In [14]:
import pandas as pd

# 'data.csv' 파일을 읽어서 DataFrame으로 변환
df = pd.read_csv('분개장_1.csv', sep=',')
print(df.head())
print('__')
print(type(df))
print('__')
print(df.columns)
print('__')
print(df.info())
print('__')
print(df.shape)


       전표일자  전표번호   계정코드    계정과목   차변금액   대변금액  거래처코드 입력사원  \
0  20230101    50  10800   외상매출금  66000      0  12282  361   
1  20230101    50  40401    제품매출      0  60000  12282  361   
2  20230101    50  25500  부가세예수금      0   6000  12282  361   
3  20230101    72  10800   외상매출금 -44000      0   5830  361   
4  20230101    72  40401    제품매출      0 -40000   5830  361   

                  입력일자  
0  2023-01-04오후3:14:08  
1  2023-01-04오후3:14:08  
2  2023-01-04오후3:14:08  
3  2023-01-04오후3:14:08  
4  2023-01-04오후3:14:08  
__
<class 'pandas.core.frame.DataFrame'>
__
Index(['전표일자', '전표번호', '계정코드', '계정과목', '차변금액', '대변금액', '거래처코드', '입력사원',
       '입력일자'],
      dtype='object')
__
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전표일자    8161 non-null   int64 
 1   전표번호    8161 non-null   int64 
 2   계정코드    8161 non-null   int64 
 3   계정과목    8161 non-null   objec

# 분개장 분석_계정수, 시간별분석

In [27]:
#가장 많이기록된 계정과목 
x=df['계정과목'].value_counts().head(n=3)
print(x)
y=df['계정과목'].value_counts(normalize=True).head(n=3)
print(y)

#특정시간에 기록된 거래수
df['입력일자_datetime']=df['입력일자'].str.replace('오전','AM').str.replace('오후','PM')
df['입력일자_datetime'] = pd.to_datetime(df['입력일자_datetime'], format='%Y-%m-%d%p%I:%M:%S')
df['입력시간'] = df['입력일자_datetime'].dt.hour
hourly_counts = df['입력시간'].value_counts().sort_index()
print(hourly_counts)

#특정시간에 기록된 거래금액
#오후 6시부터 9시(18시, 19시, 20시, 21시)까지의 거래만 필터링합니다.
# isin() 함수를 사용하여 여러 값 중 하나에 해당하는 행을 선택합니다.
filtered_df = df[df['입력시간'].isin([18, 19, 20, 21])]
# 4. 시간 범위를 변수로 설정합니다.
start_hour = 18  # 오후 6시
end_hour = 21    # 오후 9시

# 5. 연속된 시간 범위로 데이터를 필터링합니다.
# (df['입력시간'] >= start_hour) & (df['입력시간'] <= end_hour)
# 이 조건은 '입력시간'이 시작 시간 이상이고(&) 끝 시간 이하인 모든 행을 선택합니다.
filtered_df2 = df[(df['입력시간'] >= start_hour) & (df['입력시간'] <= end_hour)]

debit_sum = filtered_df['차변금액'].sum()
credit_sum = filtered_df['대변금액'].sum()

# 7. 결과를 출력합니다.
print(f" {start_hour}시부터 {end_hour}시까지의 거래 합계:")
print(f"차변금액 합계: {debit_sum:,} 원")
print(f"대변금액 합계: {credit_sum:,} 원")
total_debit=df['대변금액'].sum()
total_credit=df['차변금액'].sum()
ratio_d=debit_sum / total_debit
ratio_c=credit_sum/total_credit
print('전체대변금액 중 차지하는비율:', ratio_d)
print('전체차변금액 중 차지하는비율:', ratio_c)

계정과목
외상매출금     2615
부가세예수금    1643
제품매출      1464
Name: count, dtype: int64
계정과목
외상매출금     0.320426
부가세예수금    0.201323
제품매출      0.179390
Name: proportion, dtype: float64
입력시간
8      100
9      474
10     861
11    1347
12       4
13     557
14     618
15     717
16    1776
17     482
18     896
19      77
21     252
Name: count, dtype: int64
 18시부터 21시까지의 거래 합계:
차변금액 합계: 7,300,693,196 원
대변금액 합계: 7,257,851,196 원
전체대변금액 중 차지하는비율: 0.09345686836788629
전체차변금액 중 차지하는비율: 0.09290844384885423


# 분개장 분석_전표세트 분석

In [65]:
#전표일자와 전표번호를 기준으로 grouping
grouped_by_voucher = df.groupby(['전표일자', '전표번호'])[['차변금액', '대변금액']].sum()

#확인
print("전표별 차변/대변 금액 합계:")
print(grouped_by_voucher.head(10))
print('______')

#금액기준 상위 n개 확인 (차변기준)
sorted_by_debit_sum = grouped_by_voucher.sort_values(by='차변금액', ascending=False)
print(sorted_by_debit_sum.head(10))
print('______')

#정상적인 매출 전표세트 확인하기 (boolean indexing 활용, True인 값들만 추출)
#조건 1: 계정과목이 '보통예금'이고, (차변금액 > 0) 또는 (대변금액 < 0)인 경우
condition1_mask = (df['계정과목'] == '보통예금') & ((df['차변금액'] > 0) | (df['대변금액'] < 0))
vouchers_with_condition1 = df[condition1_mask][['전표일자', '전표번호']].drop_duplicates()
#조건 2: 계정과목이 '상품매출' 또는 '제품매출'이고, (대변금액 > 0) 또는 (차변금액 < 0)인 경우
condition2_mask = (df['계정과목'].isin(['상품매출', '제품매출'])) & ((df['대변금액'] > 0) | (df['차변금액'] < 0))
vouchers_with_condition2 = df[condition2_mask][['전표일자', '전표번호']].drop_duplicates()
#두 조건을 모두 만족하는 전표세트의 교집합
combined_vouchers = pd.merge(vouchers_with_condition1, vouchers_with_condition2, on=['전표일자', '전표번호'])
final_count = len(combined_vouchers)
print(combined_vouchers.head())
print(f"주어진 조건을 모두 만족하는 전표세트(매출)의 총 개수: {final_count}개")
print('______')

#차변 대변 불일치하는 전표세트 찾기
unbalanced_vouchers = grouped_by_voucher[grouped_by_voucher['차변금액'] != grouped_by_voucher['대변금액']]
print('불일치전표세트', '\n' , unbalanced_vouchers)
print("차액으로 2차 확인 \n")
#전체 차변금액과 대변금액의 합계를 계산합니다.
total_debit = df['차변금액'].sum()
total_credit = df['대변금액'].sum()
print(" 파일 전체 차변금액 합계:")
print(f"{total_debit:,} 원")
print("\n 파일 전체 대변금액 합계:")
print(f"{total_credit:,} 원")
print("\n 두 금액의 차이:")
print(f"{total_debit - total_credit:,} 원")
print('______')

#거래처중 레코드(~거래수)가 50개 초과하는 거래처 파악
condition_mask3 = (df['계정과목'].isin(['상품매출', '제품매출', '교육수입', '임대수입', '기타수입'])) & ((df['대변금액'] > 0) | (df['차변금액'] < 0))
frequent_customer=df[condition_mask3]['거래처코드'].value_counts()
vendors_over_50_transactions = frequent_customer[frequent_customer > 50]
print(frequent_customer)
print(vendors_over_50_transactions)
print('______')

#전표세트 사원별 입력개수 파악
#각 전표세트를 하나의 고유한 행으로 먼저 만들기_by drop_duplicates()
unique_voucher = df[['전표일자', '전표번호', '입력사원']].drop_duplicates()
#중복이 제거된 데이터에서 사원 별로 전표세트의 개수를 계산
employee_voucher_counts = unique_voucher['입력사원'].value_counts()
print(employee_voucher_counts)





전표별 차변/대변 금액 합계:
                    차변금액       대변금액
전표일자     전표번호                      
20230101 50        66000      66000
         72       -44000     -44000
         3240  520300000  520300000
20230102 3333  800800000  800800000
20230103 81        66000      66000
20230104 25       110000     110000
         179      880000     880000
         213       66000      66000
         220       66000      66000
         271       66000      66000
______
                     차변금액        대변금액
전표일자     전표번호                        
20230123 3312  1095600000  1095600000
20231202 3280  1092300000  1092300000
20230117 3302  1090100000  1090100000
20230302 3299  1087900000  1087900000
20230204 3297  1084600000  1084600000
20230807 3251  1081300000  1081300000
20230624 3320  1078000000  1078000000
20230108 3282  1074700000  1074700000
20231112 3315  1073600000  1073600000
20231011 3284  1052700000  1052700000
______
       전표일자  전표번호
0  20230101  3240
1  20230107  3261
2  20230120  3260
3  202301